# Initializing environment

In [1]:
import numpy as np
import os

# To make the output stable across other runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "training_farmer_income_models"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Loading dataset

In [3]:
import pandas as pd
train_df = pd.read_excel('Pearl Challenge data with dictionary.xlsx', sheet_name='TrainData')
test_df = pd.read_excel('Pearl Challenge data with dictionary.xlsx', sheet_name='TestData')

In [4]:
train_df.describe()

,FarmerID,Zipcode,No_of_Active_Loan_In_Bureau,Avg_Disbursement_Amount_Bureau,Non_Agriculture_Income,Total_Land_For_Agriculture,K022-Proximity to nearest mandi (Km),K022-Proximity to nearest railway (Km),KO22-Village score based on socio-economic parameters (0 to 100),K022-Seasonal Average Rainfall (mm),R022-Seasonal Average Rainfall (mm),K021-Seasonal Average Rainfall (mm),R021-Seasonal Average Rainfall (mm),R020-Seasonal Average Rainfall (mm),Perc_of_house_with_6plus_room,Women_15_19_Mothers_or_Pregnant_at_time_of_survey,perc_of_pop_living_in_hh_electricity,perc_Households_with_Pucca_House_That_Has_More_Than_3_Rooms,mat_roof_Metal_GI_Asbestos_sheets,perc_of_Wall_material_with_Burnt_brick,Households_with_improved_Sanitation_Facility,perc_Households_do_not_have_KCC_With_The_Credit_Limit_Of_50k,K022-Total Geographical Area (in Hectares)-,K022-Net Agri area (in Ha)-,K022-Net Agri area (% of total geog area)-,Kharif Seasons Irrigated area in 2022,Kharif Seasons Cropping density in 2022,Kharif Seasons Agricultural performance in 2022,Kharif Seasons Agricultural Score in 2022,Kharif Seasons Seasonal average groundwater thickness (cm) in 2022,Kharif Seasons Seasonal average groundwater replenishment rate (cm) in 2022,Rabi Seasons Season Irrigated area in 2022,Rabi Seasons Cropping density in 2022,Rabi Seasons Agricultural performance in 2022,Rabi Seasons Agricultural Score in 2022,Rabi Seasons Seasonal average groundwater thickness (cm) in 2022,Rabi Seasons Seasonal average groundwater replenishment rate (cm) in 2022,Rabi Seasons Kharif Season Irrigated area in 2021,Rabi Seasons Cropping density in 2021,Rabi Seasons Agricultural performance in 2021,Rabi Seasons Agricultural Score in 2021,Rabi Seasons Seasonal average groundwater thickness (cm) in 2021,Rabi Seasons Seasonal average groundwater replenishment rate (cm) in 2021,Kharif Seasons Kharif Season Irrigated area in 2021,Kharif Seasons Cropping density in 2021,Kharif Seasons Agricultural performance in 2021,Kharif Seasons Agricultural Score in 2021,Kharif Seasons Seasonal average groundwater thickness (cm) in 2021,Kharif Seasons Seasonal average groundwater replenishment rate (cm) in 2021,Kharif Seasons Kharif Season Irrigated area in 2020,Kharif Seasons Cropping density in 2020,Kharif Seasons Agricultural performance in 2020,Kharif Seasons Agricultural Score in 2020,Kharif Seasons Seasonal average groundwater thickness (cm) in 2020,Kharif Seasons Seasonal average groundwater replenishment rate (cm) in 2020,Rabi Seasons Kharif Season Irrigated area in 2020,Rabi Seasons Cropping density in 2020,Rabi Seasons Agricultural performance in 2020,Rabi Seasons Agricultural Score in 2020,Rabi Seasons Seasonal average groundwater thickness (cm) in 2020,Rabi Seasons Seasonal average groundwater replenishment rate (cm) in 2020,Night light index,Village score based on socio-economic parameters (Non normalised),Village score based on socio-economic parameters (0 to 100),Land Holding Index source (Total Agri Area/ no of people),Road density (Km/ SqKm),Target_Variable/Total Income
count,4.797000e+04,47970.000000,47970.000000,2.718000e+04,4.797000e+04,47899.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47802.000000,47802.000000,47802.000000,47802.000000,47802.000000,47802.000000,47802.000000,47802.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.00000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,47970.000000,4.797000e+04
mean,5.488529e+15,486844.412529,1.780196,2.463834e+05,2.8

In [5]:
print("TrainData size:",len(train_df))
print("TestData size:", len(test_df))

TrainData size: 47970
TestData size: 9986


# Preprocessing data

In [6]:
drop_cols = ['FarmerID', 'Zipcode', 'CITY', 'DISTRICT', 'VILLAGE', 'Location']
target = 'Target_Variable/Total Income'
X = train_df.drop(columns=[target] + drop_cols)
y = train_df[target]

X_test = test_df.drop(columns=[target] + drop_cols)

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

cat_cols = X.select_dtypes(include='object').columns.tolist()
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ]
)

X_preprocessed = preprocessor.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Building neural network

## 1. Simple NN

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Input dimension
input_dim = X_train.shape[1]

model = Sequential([
    Dense(128, activation='relu', input_dim=input_dim),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1)  # Output layer for regression
])

model.compile(optimizer='adam', loss='mse', metrics=['mape'])

# Train with early stopping
es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=[es],
    verbose=1
)


Epoch 1/100
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - loss: 5811745062912.0000 - mape: 93.7802 - val_loss: 4640217759744.0000 - val_mape: 31.7988
Epoch 2/100
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 4012301877248.0000 - mape: 36.8082 - val_loss: 4334829436928.0000 - val_mape: 46.5302
Epoch 3/100
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 3338294263808.0000 - mape: 47.9003 - val_loss: 4296961163264.0000 - val_mape: 49.0209
Epoch 4/100
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 4475205451776.0000 - mape: 49.9652 - val_loss: 4265132949504.0000 - val_mape: 46.9450
Epoch 5/100
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 3758292205568.0000 - mape: 48.4714 - val_loss: 4230997606400.0000 - val_mape: 48.2474
Epoch 6/100
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 4526061912064.0000 - mape: 49.5967 - val_loss: 4198045319168.0000 - val_mape: 46.9132
Epoch 7/100
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 4202643587072.0000 - mape: 47.9740 - val_l

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np

# Predict on test set
y_pred_scaled = model.predict(X_val)

# Inverse transform both prediction and true values
y_pred = y_scaler.inverse_transform(y_pred_scaled)
y_true = y_scaler.inverse_transform(y_test.reshape(-1, 1))

# Compute MAPE
real_mape = mean_absolute_percentage_error(y_true, y_pred) * 100
print(f"Real Test MAPE: {real_mape:.2f}%")


## 2. Overkill NN (using BatchNormalization+Dropout)

In [10]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def build_better_nn(input_dim):
    inputs = Input(shape=(input_dim,))
    
    x = Dense(512, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    outputs = Dense(1)(x)
    
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='mse',
                  metrics=['mape'])
    return model


In [11]:
model = build_better_nn(X_train.shape[1])

early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,
    batch_size=64,
    callbacks=[early_stop, lr_scheduler],
    verbose=1
)


Epoch 1/200
600/600 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - loss: 5915786870784.0000 - mape: 99.9997 - val_loss: 5886010982400.0000 - val_mape: 99.9985 - learning_rate: 0.0010
Epoch 2/200
600/600 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 6683652259840.0000 - mape: 99.9909 - val_loss: 5884581773312.0000 - val_mape: 99.9847 - learning_rate: 0.0010
Epoch 3/200
600/600 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 5648730292224.0000 - mape: 99.9685 - val_loss: 5882362462208.0000 - val_mape: 99.9501 - learning_rate: 0.0010
Epoch 4/200
600/600 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 6393287409664.0000 - mape: 99.9340 - val_loss: 5879887298560.0000 - val_mape: 99.9100 - learning_rate: 0.0010
Epoch 5/200
600/600 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 6773626372096.0000 - mape: 99.8887 - val_loss: 5876894138368.0000 - val_mape: 99.8560 - learning_rate: 0.0010
Epoch 6/200
600/600 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 5893537136640.0000 - mape: 99.8333 - val_loss: 5872975085568.0000 - val_

## 3. A bit more balanced NN

In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

def build_tuned_nn(input_dim):
    inputs = Input(shape=(input_dim,))
    
    x = Dense(128, activation='relu')(inputs)
    x = Dropout(0.2)(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.1)(x)
    
    outputs = Dense(1)(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='mse',
                  metrics=['mape'])
    return model

In [11]:
model = build_tuned_nn(X_train.shape[1])

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=early_stop,
    verbose=1
)

Epoch 1/100
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 5437347332096.0000 - mape: 93.1649 - val_loss: 4591176908800.0000 - val_mape: 30.3239
Epoch 2/100
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 3878629146624.0000 - mape: 36.7078 - val_loss: 4326619086848.0000 - val_mape: 47.8752
Epoch 3/100
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 3807391252480.0000 - mape: 47.9186 - val_loss: 4288770998272.0000 - val_mape: 49.5586
Epoch 4/100
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 4164289822720.0000 - mape: 49.5149 - val_loss: 4252754509824.0000 - val_mape: 48.7693
Epoch 5/100
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 5207862280192.0000 - mape: 49.5958 - val_loss: 4215910957056.0000 - val_mape: 47.3944
Epoch 6/100
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 5106872877056.0000 - mape: 49.8017 - val_loss: 4177604640768.0000 - val_mape: 46.8257
Epoch 7/100
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 4265179348992.0000 - mape: 47.2867 - val_lo